**Question 12**

Loading the iris data into pandas dataframe.

In [1]:
import pandas as pd
import numpy
irisdata = pd.read_csv("iris.data",sep=",",names=["sepal_l","sepal_w","petal_l","petal_w","_class"])
irisdata

,sepal_l,sepal_w,petal_l,petal_w,_class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


Listing unique iris flow class list

In [2]:
classes = irisdata._class.unique()

**Splitting the data for training and testing**

Creating training data and lables using first 35 records/data from each class group.

In [3]:
#training data by just picking first 35 rows from each class.
train_data = irisdata.groupby('_class').head(35)
x_train_data = train_data[["sepal_l", "sepal_w","petal_l","petal_w"]]
y_train_labels = train_data[["_class"]]


Creating test data and lables using remaining 15 records/data from each class group.

In [4]:
#test data by just picking last 15 rows from each class.
test_data = irisdata.groupby('_class').tail(15)
x_test_data = test_data[["sepal_l", "sepal_w","petal_l","petal_w"]]
y_test_labels = test_data[["_class"]]

Probabilities of each iris flower class

In [5]:
#This portion calculates the proportion or the class(iris flower type) probabilities

#initializing the dictionary to save class and its probability.
class_prob_dictionary = {}

for c in classes:
    fraction = len(y_train_labels.loc[y_train_labels['_class'] == c])/len(y_train_labels)
    class_prob_dictionary[c] = fraction
    print("The probability of iris type class "+c+" is "+str(fraction))

The probability of iris type class Iris-setosa is 0.3333333333333333
The probability of iris type class Iris-versicolor is 0.3333333333333333
The probability of iris type class Iris-virginica is 0.3333333333333333


Routine to return the matrix for a iris flower class

In [6]:
#This method gives a matrix of image for corresponding class/digit label.
def getTrainMatrixForClass(_class):
    train_data_all_columns = train_data.loc[train_data['_class'] == _class]
    train_matrix = train_data_all_columns[["sepal_l", "sepal_w","petal_l","petal_w"]]
    return train_matrix



Code to create dictionary of class and the join probability distribution.

In [8]:
import numpy as np
from scipy.stats import multivariate_normal
class_joint_distribution_dict = {} #to store class as key and pdf array for test images for that class
for c in classes:
    class_mtrx = getTrainMatrixForClass(c)  #extracting the matrix corresponding to class = c 
    _mean = class_mtrx.mean(0)
    _cov = np.cov(class_mtrx,rowvar=False)
    prob_density = multivariate_normal.logpdf(x_test_data, mean=_mean, cov=_cov)
    #calculation the joint probability Pr(y ).Pr(x |y )
    joint_distribution = np.log(class_prob_dictionary[c]) + prob_density
    class_joint_distribution_dict[c] = joint_distribution
        

Looping through all the test data to predict the class with our generative model.

In [9]:
validation_prediction = []
for idx in range(len(y_test_labels)):
    max_predict_class_dict = {}
    for c in classes:
        #print(class_joint_distribution_dict[c][idx])
        max_predict_class_dict[c] = class_joint_distribution_dict[c][idx] #picking idx th element for comparison from that class
    #print("for c: "+c+ " "+str(max_predict_class_dict))    
    class_with_max_pdf = max(max_predict_class_dict, key=max_predict_class_dict.get)
    #print("predicted: "+class_with_max_pdf)  
    validation_prediction.append((y_test_labels.iloc[[idx]]['_class'].to_string(index=False),class_with_max_pdf))
    #validation_prediction[y_test_labels.iloc[[idx]]['_class'].to_string(index=False)] = class_with_max_pdf


Routine to find the Error rate given the list of tuple(actual label and predicted label)

In [10]:
#This method takes the list of tuple(actual lable, predicted label) and calculates the error rate for that predicted set.
def calculateErrorRate(actualAndPredictedLabels):
    match=filter(lambda x: x[0] == x[1], actualAndPredictedLabels)
    nomatch=filter(lambda x: x[0] != x[1], actualAndPredictedLabels)
    match_cnt = len(list(match))
    nomatch_cnt = len(list(nomatch))
    #print("c="+str(c_val))
    return nomatch_cnt/len(y_test_labels)

**Error rate of our model**

In [11]:
error_rate = calculateErrorRate(validation_prediction)
print("Error rate with the training data and the model = "+str(error_rate)+" %")

Error rate with the training data and the model = 0.0 %
